In [1]:
from liberty.parser import parse_liberty
from liberty.types import select_cell
import os
from subprocess import call
from glob import glob
import shutil

def clear_dir(target_dir):
    if(os.path.isdir(target_dir)):
        shutil.rmtree(target_dir)
    os.mkdir(target_dir)

In [3]:
# Cells we want to copy as custom macros
snip_cells = ["sky130_fd_sc_hd__and2_1", "sky130_fd_sc_hd__fa_1", "sky130_fd_sc_hd__edfxtp_1"]
std_cell_prefix = "tt_export_"

# Location of liberty files
PDK_ROOT = os.environ.get('PDK_ROOT')
TECH="sky130_fd_sc_hd"
lib_regex = f"{PDK_ROOT}/sky130A/libs.ref/sky130_fd_sc_hd/lib/*.lib" 
lib_files = glob(lib_regex)

In [3]:
# Lefs can be done manually, shared file, pretty easy to copy and paste
# TODO find a good LEF parser and parse the library as is done below

In [4]:
# 
clear_dir('./lib')
for lib_file in lib_files:
    out_name='./lib/' + std_cell_prefix + os.path.basename(lib_file)
    lib_txt = open(lib_file).read()
    lib = parse_liberty(lib_txt)
    # Crappy way to get the library header TODO: properly parse out this information
    lib_header = lib_txt.split("cell (")[0]
    lib_cell_txt = []
    for cell in snip_cells:
        lib_cell_txt.append(str(select_cell(lib, cell)))
    new_lib_str=[lib_header] + lib_cell_txt
    new_lib_str = "\n".join(new_lib_str) + "\n}\n"
    new_lib_str = new_lib_str.replace(TECH, std_cell_prefix+TECH)
    with open(f'{out_name}', 'w') as f_lib:
        f_lib.write(new_lib_str)

In [8]:
# Build a Klayout script that reads in a GDS, removes non-target cells and prefixes target cells

# Get EVs
OPENLANE_ROOT=os.getenv('OPENLANE_ROOT')
PDK_ROOT=os.getenv('PDK_ROOT')
USER=os.getenv('USER')
UID=os.geteuid()
GID=os.getegid()
CWD=os.getcwd()
LIB_PDK="sky130A/libs.ref"
TECH="sky130_fd_sc_hd"
# Tech path
TECH_PATH=f"{PDK_ROOT}/{LIB_PDK}/{TECH}"

clear_dir('./gds')

# Get GDS file paths
GDS_FILES=glob(TECH_PATH+f"/gds/{TECH}*.gds")

kl_script = []

cell_str = []
for cell in snip_cells:
    cell_str.append(f'{cell}')
cells_str = ', '.join(cell_str)
cells_str = f'[{cells_str}]'
print(cells_str)

kl_script.append(f'prefix_str="{std_cell_prefix}*"')
kl_script.append(f"target_cells = {cell_str}")
kl_script.append("target_cells.each do |targ_cell|")
kl_script.append("\tprune_cells = []")
kl_script.append("\tlayout = RBA::Layout::new")
kl_script.append("\tlayout.read($input)")
kl_script.append("\tlayout.each_cell do |cell|")
kl_script.append("\t\tcell_name = layout.cell_name(cell.cell_index)")
kl_script.append("\t\tif targ_cell == cell_name")
kl_script.append('\t\t\tputs "Found " + cell_name')
kl_script.append("\t\t\tnew_cell_name = prefix_str.gsub(/#/, cell.cell_index.to_s).gsub(/\\*/, cell_name)")
kl_script.append("\t\t\tlayout.rename_cell(cell.cell_index, new_cell_name)")
kl_script.append("\t\telse")
kl_script.append('\t\t\tputs "Other cell " + cell_name')
kl_script.append("\t\t\tprune_cells = prune_cells + [cell_name]")
kl_script.append("\t\tend")
kl_script.append("\tend")
kl_script.append("\tprune_cells.each do |cell|")
kl_script.append('\t\tputs "Pruning " + cell')
kl_script.append("\t\tlayout.prune_cell(layout.cell_by_name(cell), -1)")
kl_script.append("\tend")
kl_script.append('\tnew_cell_name = prefix_str.gsub(/\*/, "") + targ_cell')
kl_script.append('\tlayout.write($output_dir + new_cell_name + ".gds")')
kl_script.append("end")

# Write script to CWD - this is the project directory in the OpenLane container
kl_script_name="klayout_script.rb"
with open(kl_script_name, "w") as f_kl:
    f_kl.write("\n".join(kl_script))

# Construct docker klayout command
klayout_cmd_run=[]
klayout_cmd_run.append('docker')
klayout_cmd_run.append('run')
klayout_cmd_run.append(f'-v {OPENLANE_ROOT}:/openLANE_flow')
klayout_cmd_run.append(f'-v {PDK_ROOT}:{PDK_ROOT}')
klayout_cmd_run.append(f'-v {CWD}:/project') 
klayout_cmd_run.append(f'-v {CWD}/gds:/project/gds') 
klayout_cmd_run.append(f'-e PDK_ROOT={PDK_ROOT}')
klayout_cmd_run.append(f'-u {UID}:{GID}')
klayout_cmd_run.append(f'efabless/openlane')

for f_gds in GDS_FILES:
    out_name=std_cell_prefix + os.path.basename(f_gds)
    kl_cmd=f'klayout -b -r /project/{kl_script_name} -rd input={f_gds} -rd output_dir=/project/gds/ -l {PDK_ROOT}/sky130A/libs.tech/klayout/tech/sky130A.lyp -nn {PDK_ROOT}/sky130A/libs.tech/klayout/tech/sky130A.lyt'
    with open("./docker_output.log", "w") as output:
        call(" ".join(klayout_cmd_run + [kl_cmd]), shell=True, stdout=output, stderr=output)
    print("Docker running command:\n" + " ".join(klayout_cmd_run + [kl_cmd]))

# Command to check GDS afterward:
# docker run -it -v $OPENLANE_ROOT:/openLANE_flow -v $PDK_ROOT:$PDK_ROOT -v $PWD/gds:/project -e PDK_ROOT=$PDK_ROOT -v /tmp/.X11-unix:/tmp/.X11-unix -e DISPLAY=$DISPLAY -e QT_GRAPHICSSYSTEM="native" -u $(id -u $USER):$(id -g $USER) efabless/openlane klayout -s /project/tt_export_sky130_fd_sc_hd.gds -l $PDK_ROOT/sky130A/libs.tech/klayout/tech/sky130A.lyp


[sky130_fd_sc_hd__and2_1, sky130_fd_sc_hd__fa_1, sky130_fd_sc_hd__edfxtp_1]
Docker running command:
docker run -v /opt/tt/openlane:/openLANE_flow -v /opt/tt/pdk:/opt/tt/pdk -v /mada/users/tsheaves/Desktop/tt06_hsc_tdc/src/std_cell_macros:/project -v /mada/users/tsheaves/Desktop/tt06_hsc_tdc/src/std_cell_macros/gds:/project/gds -e PDK_ROOT=/opt/tt/pdk -u 7547:1211 efabless/openlane klayout -b -r /project/klayout_script.rb -rd input=/opt/tt/pdk/sky130A/libs.ref/sky130_fd_sc_hd/gds/sky130_fd_sc_hd.gds -rd output_dir=/project/gds/ -l /opt/tt/pdk/sky130A/libs.tech/klayout/tech/sky130A.lyp -nn /opt/tt/pdk/sky130A/libs.tech/klayout/tech/sky130A.lyt
